<a href="https://www.kaggle.com/code/mohitjhedu/practice-1?scriptVersionId=193049396" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Install and import the necessary libabries

In [ ]:
pip install torch torchvision

In [ ]:
pip install yolov5


In [ ]:
# import necessary libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import shutil
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Unzip the dataset

In [ ]:
# unzip orignal data file from drive 

! unzip '/content/drive/MyDrive/Colab Notebooks/dog bread classsifier/dog-breed-identification.zip'

## Load the Lables DataSet

In [ ]:
# load the label csv file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dog bread classsifier/labels.csv')

In [ ]:
breed_s = df.breed.value_counts()

In [ ]:
breed_s

## Yolo Model Testing

In [ ]:
import torch
from yolov5 import YOLOv5

# Load a pre-trained YOLOv5 model
model = YOLOv5("yolov5s.pt", device="cuda" if torch.cuda.is_available() else "cpu")

## Functions

In [ ]:
def cropped_image(image, model, threshold=0.5):
  result = model.predict(image)
  boxes = result.xyxy[0].numpy()

  for box in boxes:
    x1, y1, x2, y2, conf, cls = box
    class_name = result.names.get(int(cls))

    if class_name == 'dog':
      cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
      return cropped_image

In [ ]:
def size_reshaper(img):
  height, weight = img.shape[:2]

  if height > weight:
    pad = (height - weight)
    left = pad // 2
    right = pad - left
    top = 0
    bottom = 0
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value = [255, 255, 255])

  elif weight > height:
    pad = (weight - height)
    left = 0
    right = 0
    top = pad // 2
    bottom = pad - top
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value = [255, 255, 255])

  else:
    return img



### Function That make the Dataset in the Correct Formet

In [ ]:
def data_prep(path, label_path, model, threshold = 0.5, train_size = 0.8):

  # get the data
  total_data = os.listdir(path)

  # get the the labels
  df = pd.read_csv(label_path)
  df.set_index('id', inplace=True)


  # devide the data into train and test
  train_row = int(len(total_data) * train_size)


  # devide the data into train and test
  train_data = total_data[:train_row]
  test_data = total_data[train_row:]

  # make the dirs of train and test
  os.makedirs('/content/data_split/train', exist_ok=True)
  os.makedirs('/content/data_split/test', exist_ok=True)

  train_path = '/content/data_split/train'
  test_path = '/content/data_split/test'


  # move the data into train_dir
  for file in train_data:

    # get the file id
    file_id = file.split('.')[0]
    if file_id not in df.index:
      continue

    dog_org = df.loc[file_id].values[0]
    image_path = os.path.join(path, file)
    img = cv2.imread(image_path)

    # make the new dir in train of dog label
    dir_name = os.path.join(train_path, dog_org)
    os.makedirs(dir_name, exist_ok=True)

    # cropped_image
    result = model.predict(img)
    boxes = result.xyxy[0].numpy()

    for i,box in enumerate(boxes):
      x1, y1, x2, y2, conf, cls = box
      class_name = result.names.get(int(cls))

      if class_name == 'dog':
        cropped_image = img[int(y1):int(y2), int(x1):int(x2)].copy()
        new_image = size_reshaper(cropped_image)
        cv2.imwrite(os.path.join(dir_name,f'{file_id}.jpg'), new_image)


  # move the data into test_dir
  for file in test_data:

    # get the file id
    file_id = file.split('.')[0]
    if file_id not in df.index:
      continue

    dog_org = df.loc[file_id].values[0]
    image_path = os.path.join(path, file)
    img = cv2.imread(image_path)

    # make the new dir in test of dog label
    dir_name = os.path.join(test_path, dog_org)
    os.makedirs(dir_name, exist_ok=True)

    # cropped_image
    result = model.predict(img)
    boxes = result.xyxy[0].numpy()

    for i,box in enumerate(boxes):
      x1, y1, x2, y2, conf, cls = box
      class_name = result.names.get(int(cls))

      if class_name == 'dog':
        cropped_image = img[int(y1):int(y2), int(x1):int(x2)].copy()
        new_image = size_reshaper(cropped_image)
        cv2.imwrite(os.path.join(dir_name,f'{file_id}.jpg'), new_image)

  print('data is ready')

In [ ]:
data_prep('/content/train','/content/labels.csv',model, train_size = 0.7)

## Zip File Created, and move to Drive

In [ ]:
# created zip and move the data_split_new zip to drive
shutil.make_archive('data_split_new', 'zip', '/content/data_split')
shutil.move('/content/data_split_new.zip', '/content/drive/MyDrive/Colab Notebooks/dl/dog bread classsifier')


## unzip data_split Folder

In [ ]:
! unzip '/content/drive/MyDrive/Colab Notebooks/dl/dog bread classsifier/data_split_new.zip'


### Data Analysis

In [ ]:
# train height and weight 

shape = []
path = '/kaggle/input/dog-bread-classifier/data_split_new/train'
for folder in os.listdir('/kaggle/input/dog-bread-classifier/data_split_new/train'):
    new_path = os.path.join(path,folder)
    for image_path in os.listdir(new_path):
        image_path_ = os.path.join(new_path,image_path)
        image = cv2.imread(image_path_)
        shape.append(image.shape[:2])
                
    

In [ ]:
# test height and weight
path = '/kaggle/input/dog-bread-classifier/data_split_new/test'
for folder in os.listdir('/kaggle/input/dog-bread-classifier/data_split_new/test'):
    new_path = os.path.join(path,folder)
    for image_path in os.listdir(new_path):
        image_path_ = os.path.join(new_path,image_path)
        image = cv2.imread(image_path_)
        shape.append(image.shape[:2])

In [ ]:
shape_df = pd.DataFrame(shape, columns = ['height', 'weight'])

In [ ]:
shape_df.mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(data = shape_df, x = 'height')
# vartical line on x axis
plt.axvline(shape_df['height'].mean(), color = 'r')
plt.axvline(shape_df['height'].median(), color = 'b')
plt.axvline(shape_df['height'].max(), color = 'b')


In [ ]:
sns.boxplot(data = shape_df, x = 'height')

In [ ]:
q1= np.percentile(shape_df['height'], 25)
q3 = np.percentile(shape_df['height'], 75)
iqr = q3 - q1
ub = q3 + (1.5*iqr)

tp =shape_df[shape_df.height <= ub]

sns.kdeplot(data = tp, x = 'height')

In [ ]:
sns.kdeplot(data = tp, x = 'weight')

In [ ]:
len(shape_df) - len(tp)

In [ ]:
# tp.mean()
shape_df.mean()

### Data Generater to load the Data from Directory

In [ ]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/dog-bread-classifier/data_split_new/train',
    labels='inferred',
    label_mode='int',
    batch_size=150,
    image_size=(320, 320),
    shuffle=False)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/dog-bread-classifier/data_split_new/test',
    labels='inferred',
    label_mode='int',
    batch_size=150,
    image_size=(320, 320),
    shuffle = False
    )

In [ ]:
# apply agumention and preprocssing to the trin_Set
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomBrightness(0.2)
])

from tensorflow.keras.applications.resnet50 import preprocess_input


def aug_preprocess(image, label):

  image_aug = data_augmentation(image)
  n_image = preprocess_input(image_aug)
  return n_image, label

def preprocess(image, label):
  n_image = preprocess_input(image)
  return n_image, label

train_set_pre = train_set.map(aug_preprocess)
test_set_pre = test_set.map(preprocess)


## Fine tune a Model

In [ ]:
base_model = tf.keras.applications.ResNet50(weights = 'imagenet',include_top=False, input_shape=(320, 320, 3))

In [ ]:
for i,layer in enumerate(base_model.layers):
  if i > 165:
    layer.trainable = True
    print(layer.trainable)
  else:
    layer.trainable = False


In [ ]:
print(len(base_model.layers))

In [ ]:
# build a model

# model = tf.keras.Sequential([
#   tf.keras.layers.Input(shape=(320, 320, 3)),
    
#     tf.keras.layers.Conv2D(20,(3,3),padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    
#     tf.keras.layers.Conv2D(30,(3,3),padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    
#     tf.keras.layers.Conv2D(35,(3,3),padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    
#     tf.keras.layers.Conv2D(40,(3,3),padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    
#     tf.keras.layers.Conv2D(40,(3,3),padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(300, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     tf.keras.layers.Dense(200, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     tf.keras.layers.Dense(100, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     tf.keras.layers.Dense(50, activation = 'relu'),
#     tf.keras.layers.Dense(120, activation='softmax')
# ])

# model.summary()

from tensorflow.keras.layers import Dense,Input, Conv2D, MaxPool2D, Flatten, AveragePooling2D,BatchNormalization, Dropout
from tensorflow.keras.models import Model

Model_op = base_model.output
Model_op = AveragePooling2D(pool_size = (4,4))(Model_op)
Model_op = Flatten()(Model_op)

Model_op = Dense(256, activation='relu')(Model_op)
Model_op = BatchNormalization()(Model_op)
Model_op = Dropout(0.4)(Model_op)

Model_op = Dense(50, activation = 'relu')(Model_op)
Model_op = Dense(120, activation='softmax')(Model_op)

model = Model(inputs = base_model.input, outputs = Model_op)

model.summary()

In [ ]:
# set early stopping callback
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

check_point = ModelCheckpoint(filepath = '/kaggle/working/epoch{epoch:02d}.keras',
                              save_best_only=True,
                              monitor='val_accuracy',
                              mode = 'max',
                             )

early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10, mode='auto')
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.004)

model.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_set_pre, validation_data=test_set_pre,batch_size = 150 ,epochs= 100, callbacks = [early_stop, check_point])